## 노드 재시도 정책을 추가하는 방법
API를 호출하거나, 데이터베이스를 쿼리하거나, LLM을 호출하는 경우 등 노드에 사용자 지정 재시도 정책이 필요한 경우

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

재시도 정책을 설정하려면 add_node 함수에 `retry` 매개변수를 전달해야 합니다.

`retry` 매개변수는 `RetryPolicy`라는 이름 있는 튜플(named tuple) 객체를 입력으로 받습니다.

아래는 기본 매개변수로 `RetryPolicy` 객체를 생성하는 예시입니다.

In [2]:
from langgraph.pregel import RetryPolicy

RetryPolicy()

RetryPolicy(initial_interval=0.5, backoff_factor=2.0, max_interval=128.0, max_attempts=3, jitter=True, retry_on=<function default_retry_on at 0x106769080>)

기본적으로 `retry_on` 매개변수는 `default_retry_on` 함수를 사용합니다.

이 함수는 다음 예외들을 제외한 모든 예외에 대해 재시도를 수행합니다:
- `ValueError`
- `TypeError`
- `ArithmeticError`
- `ImportError`
- `LookupError`
- `NameError`
- `SyntaxError`
- `RuntimeError`
- `ReferenceError`
- `StopIteration`
- `StopAsyncIteration`
- `OSError`
또한, `requests`나 `httpx` 같은 인기 있는 HTTP 요청 라이브러리에서 발생하는 예외의 경우,
5xx 상태 코드(서버 오류) 에 대해서만 재시도를 수행합니다.

### 노드에 재시도 정책 전달
마지막으로, add_node 함수를 호출할 때 `RetryPolicy` 객체를 함께 전달할 수 있습니다.

아래 예제에서는 각각의 노드에 두 가지 서로 다른 재시도 정책을 전달하는 모습을 보여줍니다.

In [ ]:
from langchain_openai import ChatOpenAI
import operator
import sqlite3
from typing import Annotated, Sequence
from typing_extensions import TypedDict

from langchain_anthropic import ChatAnthropic
from langchain_core.messages import BaseMessage

from langgraph.graph import END, StateGraph, START
from langchain_community.utilities import SQLDatabase
from langchain_core.messages import AIMessage

db = SQLDatabase.from_uri("sqlite:///:memory:")

# model = ChatAnthropic(model_name="claude-2.1")
model = ChatOpenAI(model="gpt-4o-mini")

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]


def query_database(state):
    query_result = db.run("SELECT * FROM Artist LIMIT 10;")
    return {"messages": [AIMessage(content=query_result)]}


def call_model(state):
    response = model.invoke(state["messages"])
    return {"messages": [response]}


# Define a new graph
builder = StateGraph(AgentState)
builder.add_node(
    "query_database",
    query_database,
    retry=RetryPolicy(retry_on=sqlite3.OperationalError),
)
builder.add_node("model", call_model, retry=RetryPolicy(max_attempts=5))
builder.add_edge(START, "model")
builder.add_edge("model", "query_database")
builder.add_edge("query_database", END)

graph = builder.compile()